In [0]:
# Importamos bibliotecas necesarias
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Cargamos el conjunto de datos
df = pd.read_csv("https://raw.githubusercontent.com/Fibovin/des_modelos_1/refs/heads/main/Student_Performance.csv")

# Codificamos la variable categórica
df['Extracurricular Activities'] = df['Extracurricular Activities'].map({'Yes': 1, 'No': 0})

# Dividimos el conjunto de datos en características y variable objetivo
X = df.drop(columns=['Performance Index'])
y = df['Performance Index']

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuramos el experimento de MLflow
mlflow.set_experiment("/Users/jf.neiraf1@uniandes.edu.co/Student_Performance")

# Definimos y ejecutamos el experimento con MLflow
with mlflow.start_run(run_name="Regresion ARBOL DECISION Experimento_1"):
    # Creamos el modelo de regresión con árbol de decisión (sin parámetros adicionales)
    tree_model = DecisionTreeRegressor(random_state=42)
    tree_model.fit(X_train, y_train)

    # Realizamos predicciones en el conjunto de prueba
    y_test_pred_tree = tree_model.predict(X_test)

    # Registramos el modelo en MLflow
    mlflow.sklearn.log_model(tree_model, "decision-tree-model")
    

    # Calculamos y registramos las métricas de interés
    r2_test_tree = r2_score(y_test, y_test_pred_tree)
    mse_test_tree = mean_squared_error(y_test, y_test_pred_tree)
    mae_test_tree = mean_absolute_error(y_test, y_test_pred_tree)
    
    mlflow.log_metric("r2_score", r2_test_tree)
    mlflow.log_metric("mse", mse_test_tree)
    mlflow.log_metric("mae", mae_test_tree)
    
    # Imprimimos las métricas para verificar
    print(f"R²: {r2_test_tree}")
    print(f"Mean Squared Error: {mse_test_tree}")
    print(f"Mean Absolute Error: {mae_test_tree}")


2024/11/10 00:57:51 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

R²: 0.9762195132401258
Mean Squared Error: 8.812694444444444
Mean Absolute Error: 2.3378333333333337


## **Tunning del modelo - Regresión con árbol de decisión**

### **Busqueda de hiperparametros con Grid search CV**

In [0]:
#Definimos el espacio de búsqueda de hiperparámetros
param_grid = {'max_depth': [3, 5, 10, None],'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4]}

#Configuramos la búsqueda de hiperparámetros en el árbol de decisión
grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Resultados
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor R² obtenido:", grid_search.best_score_)

Mejores hiperparámetros: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
Mejor R² obtenido: 0.9830783642033294


### VAMOS A EJECUTAR EL SEGUNDO EXPERIMENTO DEL MODELO TUNEADO CON BASE A LOS PARÁMETROS ENCONTRADOS ANTERIORMENTE

In [0]:
# Configuramos el experimento de MLflow
mlflow.set_experiment("/Users/jf.neiraf1@uniandes.edu.co/Student_Performance")

# Definimos y ejecutamos el experimento con MLflow para el modelo optimizado
with mlflow.start_run(run_name="Regresion ARBOL DECISION Experimento_2"):
    # Creamos el modelo con los mejores hiperparámetros
    optimized_tree_model = DecisionTreeRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10, random_state=42)
    optimized_tree_model.fit(X_train, y_train)

    # Realizamos predicciones en el conjunto de prueba
    y_test_pred_optimized = optimized_tree_model.predict(X_test)

    # Calculamos y registramos las métricas para el conjunto de prueba
    r2_test_optimized = r2_score(y_test, y_test_pred_optimized)
    mse_test_optimized = mean_squared_error(y_test, y_test_pred_optimized)
    mae_test_optimized = mean_absolute_error(y_test, y_test_pred_optimized)
    
    mlflow.log_metric("r2_score", r2_test_optimized)
    mlflow.log_metric("mse", mse_test_optimized)
    mlflow.log_metric("mae", mae_test_optimized)

    # Registramos el modelo optimizado en MLflow
    mlflow.sklearn.log_model(optimized_tree_model, "decision-tree-model")

    # Imprimimos las métricas para verificar
    print("Conjunto de Prueba:")
    print(f"R²: {r2_test_optimized}")
    print(f"Mean Squared Error: {mse_test_optimized}")
    print(f"Mean Absolute Error: {mae_test_optimized}")

2024/11/10 01:14:46 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.4/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Conjunto de Prueba:
R²: 0.9842770387732579
Mean Squared Error: 5.8266954100757244
Mean Absolute Error: 1.9034931306771197
